<a href="https://colab.research.google.com/github/newmantic/capped_variance_swap/blob/main/capped_variance_swap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class CappedVarianceSwap:
    def __init__(self, initial_price, strike_variance, variance_cap, maturity, risk_free_rate):
        self.initial_price = initial_price
        self.strike_variance = strike_variance
        self.variance_cap = variance_cap
        self.maturity = maturity
        self.risk_free_rate = risk_free_rate

    def simulate_price_path(self, n_steps, n_scenarios, volatility):
        dt = self.maturity / n_steps
        price_paths = np.zeros((n_steps + 1, n_scenarios))
        price_paths[0] = self.initial_price

        for t in range(1, n_steps + 1):
            z = np.random.standard_normal(n_scenarios)
            price_paths[t] = price_paths[t - 1] * np.exp((self.risk_free_rate - 0.5 * volatility ** 2) * dt + volatility * np.sqrt(dt) * z)

        return price_paths

    def calculate_realized_variance(self, price_path):
        log_returns = np.diff(np.log(price_path), axis=0)
        realized_variance = np.mean(log_returns ** 2) * 252  # Annualize variance assuming 252 trading days
        return min(realized_variance, self.variance_cap)

    def calculate_payoff(self, price_paths):
        realized_variances = np.array([self.calculate_realized_variance(price_paths[:, i]) for i in range(price_paths.shape[1])])
        return np.mean(realized_variances - self.strike_variance)

    def run_simulation(self, n_steps=252, n_scenarios=10000, volatility=0.2):
        price_paths = self.simulate_price_path(n_steps, n_scenarios, volatility)
        payoff = self.calculate_payoff(price_paths)
        return payoff

# Example usage
if __name__ == "__main__":
    swap = CappedVarianceSwap(
        initial_price=100,
        strike_variance=0.04,  # 20% squared
        variance_cap=0.09,      # 30% squared
        maturity=1,
        risk_free_rate=0.02
    )

    expected_payoff = swap.run_simulation()
    print(f"Expected Payoff: {expected_payoff:.4f}")

Expected Payoff: -0.0001
